# How to use different estimators and how to interpret the results

In [1]:
import pandas  as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.optimization as opt
from biogeme.expressions import Beta, DefineVariable
import seaborn as sns
import matplotlib.pyplot as plt

**Import Swissmetro data**

In [2]:
pandas = pd.read_csv("../../Data/6-Discrete Choice Models/swissmetro.dat",sep='\t')
database = db.Database("data/swissmetro", pandas)

**Use collumn names as variables**

In [3]:
globals().update(database.variables)

**Exclude some unwanted entries**

In [4]:
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0

database.remove(exclude)

**Define some dummy variables**

In [5]:
SM_COST = SM_CO * ( GA == 0 )
TRAIN_COST = TRAIN_CO * ( GA == 0 )

CAR_AV_SP = DefineVariable ('CAR_AV_SP', CAR_AV * ( SP !=0 ), database)
TRAIN_AV_SP = DefineVariable ('TRAIN_AV_SP', TRAIN_AV * ( SP != 0 ), database)

**Rescale some data**

In [6]:
TRAIN_TT_SCALED   = DefineVariable('TRAIN_TT_SCALED',   TRAIN_TT / 100.0, database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100, database)
SM_TT_SCALED      = DefineVariable('SM_TT_SCALED',      SM_TT / 100.0   , database)
SM_COST_SCALED    = DefineVariable('SM_COST_SCALED',    SM_COST / 100   , database)
CAR_TT_SCALED     = DefineVariable('CAR_TT_SCALED',     CAR_TT / 100    , database)
CAR_CO_SCALED     = DefineVariable('CAR_CO_SCALED',     CAR_CO / 100    , database)

**Create parameters to be estimated**

In [7]:
ASC_CAR = Beta('ASC_CAR',0,None ,None ,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None ,None ,0)
ASC_SM = Beta('ASC_SM',0,None ,None ,1)
B_TIME = Beta('B_TIME',0,None ,None ,0)
B_COST = Beta('B_COST',0,None ,None ,0)

**Define the utility functions**

In [8]:
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

**Associate utility functions with alternatives and associate availability of alternatives**

In [9]:
V = {1: V1,
     2: V2,
     3: V3}

av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

**Define the model**

In [10]:
logprob = models.loglogit(V, av, CHOICE)

**Define the Biogeme object**

In [11]:
biogeme  = bio.BIOGEME(database, logprob)

biogeme.modelName = "swissmetro_logit_estimators"

**Define the algorithms to estimat the maximum likelihood**

In [12]:
algos = {'CFSQP                ': None,
         'scipy                ': opt.scipy,
         'Trust region (dogleg)': opt.newtonTrustRegionForBiogeme,
         'Trust region (cg)    ': opt.newtonTrustRegionForBiogeme,
         'TR-BFGS              ': opt.bfgsTrustRegionForBiogeme,
         'Simple bounds Newton ': opt.simpleBoundsNewtonAlgorithmForBiogeme,
         'Simple bounds BFGS   ': opt.simpleBoundsNewtonAlgorithmForBiogeme,
         'Simple bounds hybrid ': opt.simpleBoundsNewtonAlgorithmForBiogeme}

algoParameters = {'Trust region (dogleg)': {'dogleg':True},
                  'Trust region (cg)': {'dogleg':False},
                  'Simple bounds Newton ': {'proportionAnalyticalHessian': 1.0},
                  'Simple bounds BFGS   ': {'proportionAnalyticalHessian': 0.0},
                  'Simple bounds hybrid ': {'proportionAnalyticalHessian': 0.5}}

**Estimate the model**

In [13]:
biogeme.generateHtml = False
biogeme.generatePickle = False

results = {}
for name,algo in algos.items():
    p = algoParameters.get(name)
    results[name] = biogeme.estimate(algorithm=algo,algoParameters=p)
    g = results[name].data.g
    biogeme.createLogFile()

**Print results for the different algorithms**

In [14]:
print("Algorithm\t\tloglike\t\tnormg\ttime")
print("+++++++++\t\t+++++++\t\t+++++\t++++")
for name,algo in algos.items():
    print(f'{name}\t{results[name].data.logLike:.2f}\t'
          f'{results[name].data.gradientNorm:.2g}\t'
          f'{results[name].data.optimizationMessages["Optimization time"]}')

Algorithm		loglike		normg	time
+++++++++		+++++++		+++++	++++
CFSQP                	-5331.25	0.0004	0:00:00.279782
scipy                	-5331.25	6.5e-05	0:00:00.085049
Trust region (dogleg)	-5331.25	0.03	0:00:00.070050
Trust region (cg)    	-5331.25	0.03	0:00:00.068124
TR-BFGS              	-5331.25	0.013	0:00:00.251144
Simple bounds Newton 	-5331.25	0.00063	0:00:00.069949
Simple bounds BFGS   	-5331.25	0.015	0:00:00.269823
Simple bounds hybrid 	-5331.25	0.00038	0:00:00.081564


**Print results**

In [15]:
results_1 = results['scipy                ']
results_2 = results['CFSQP                ']

betas_1 = results_1.getBetaValues()
betas_2 = results_2.getBetaValues()

for k in betas_1.keys():
    b_1 = betas_1[k]
    b_2 = betas_2[k]
    print(f"{k:10}=\t{b_1:.8g}\t{b_2:.8g}")

ASC_CAR   =	-0.15463245	-0.15463283
ASC_TRAIN =	-0.70118664	-0.70118664
B_COST    =	-1.0837907	-1.0837909
B_TIME    =	-1.2778603	-1.2778601


**Explore the results**

In [16]:
results = results['scipy                ']

Results = results.getEstimatedParameters()
display(Results)

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,-0.154632,0.043235,-3.576518,0.000348,0.058163,-2.658586,0.007847
ASC_TRAIN,-0.701187,0.054874,-12.778137,0.000000,0.082562,-8.492846,0.000000
B_COST,-1.083791,0.051830,-20.910412,0.000000,0.068225,-15.885522,0.000000
B_TIME,-1.277860,0.056883,-22.464577,0.000000,0.104254,-12.257125,0.000000


**Get the general statistics**
* Number of estimated parameters($K$)
* Sample size ($N$)
* Number of excluded observations
* Log likelihood of the sample with the default values for the parameters ($\mathcal{L}^i)$)
* Log likelihood for the final estimated model ($\mathcal{L}^*)$)
* Likelihood ratio:
\begin{align}
-2 (\mathcal{L}^i-\mathcal{L}^*)
\end{align}
* Rho-square for the init model
\begin{align}
\rho^2 = 1- \frac{\mathcal{L}^*}{\mathcal{L}^i}
\end{align}
* Rho-square adjusted for the init model
\begin{align}
\rho^2 = 1- \frac{\mathcal{L}^* - K}{\mathcal{L}^i}
\end{align}
* Akaike Information Criterion
\begin{align}
2 K - 2 \mathcal{L}^*
\end{align}
* Bayesian Information Criterion
\begin{align}
K N- 2 \mathcal{L}^*
\end{align}
* Final gradient norm
* Iterations
* Optimization Time

In [17]:
gs = results.getGeneralStatistics()

for k,v in gs.items():
    print("{}= {}".format(k.ljust(45),v[0]))

Number of estimated parameters               = 4
Sample size                                  = 6768
Excluded observations                        = 3960
Init log likelihood                          = -6964.662979192295
Final log likelihood                         = -5331.252006915832
Likelihood ratio test for the init. model    = 3266.8219445529267
Rho-square for the init. model               = 0.23452835796311466
Rho-square-bar for the init. model           = 0.23395403010088356
Akaike Information Criterion                 = 10670.504013831664
Bayesian Information Criterion               = 10697.783857436807
Final gradient norm                          = 6.477959424450974e-05
Nbr of threads                               = 8
